<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TPUs_in_Colab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

#Data process

In [ ]:
diff --git a/mindspore/lite/src/dequant.h b/mindspore/lite/src/dequant.h
index 114cb4f..f20e3ce 100644
--- a/mindspore/lite/src/dequant.h
+++ b/mindspore/lite/src/dequant.h
@@ -40,23 +40,19 @@ class DequantUtil {
   static void RestoreTensorData(const std::map<Tensor *, std::pair<TypeId, void *>> &tensor_origin_data_map);
 
   template <typename ST, typename DT = float>
-  static bool IsPerBatch(lite::Tensor *input_tensor, ST *quant_datas, DT *dequant_datas) {
-    if (input_tensor->shape().size() == kPerBatch &&
-        input_tensor->quant_params().size() == static_cast<size_t>(input_tensor->shape().at(0))) {  // per batch matmul
-      auto per_batch_size = input_tensor->shape().at(0);
-      auto quant_param = input_tensor->quant_params();
-      for (int i = 0; i < per_batch_size; i++) {
-        auto param = quant_param.at(i);
-        auto scale = param.scale;
-        auto zero_point = param.zeroPoint;
-        auto matrix_size = input_tensor->ElementsNum() / per_batch_size;
-        for (int64_t j = 0; j < matrix_size; j++) {
-          dequant_datas[i * matrix_size + j] = static_cast<DT>((quant_datas[i * matrix_size + j] - zero_point) * scale);
-        }
+  static DT *DequantPerBatch(lite::Tensor *input_tensor, ST *quant_datas, DT *dequant_datas) {
+    auto per_batch_size = input_tensor->shape().at(0);
+    auto quant_param = input_tensor->quant_params();
+    for (int i = 0; i < per_batch_size; i++) {
+      auto param = quant_param.at(i);
+      auto scale = param.scale;
+      auto zero_point = param.zeroPoint;
+      auto matrix_size = input_tensor->ElementsNum() / per_batch_size;
+      for (int64_t j = 0; j < matrix_size; j++) {
+        dequant_datas[i * matrix_size + j] = static_cast<DT>((quant_datas[i * matrix_size + j] - zero_point) * scale);
       }
-      return true;
     }
-    return false;
+    return dequant_datas;
   }
   template <typename ST, typename DT = float>
   static DT *DequantPerChannel(lite::Tensor *input_tensor, ST *quant_datas, DT *dequant_datas, bool channel_first) {
@@ -109,7 +105,10 @@ class DequantUtil {
       MS_LOG(ERROR) << "Malloc failed.";
       return nullptr;
     }
-    if (!IsPerBatch(input_tensor, quant_datas, dequant_datas) && input_tensor->quant_params().size() != kPerTensor) {
+    if (input_tensor->shape().size() == kPerBatch &&
+        input_tensor->quant_params().size() == static_cast<size_t>(input_tensor->shape().at(0))) {  // per batch matmul
+      return DequantPerBatch(input_tensor, quant_datas, dequant_datas);
+    } else if (input_tensor->quant_params().size() != kPerTensor) {
       return DequantPerChannel(input_tensor, quant_datas, dequant_datas, channel_first);
     } else {
       auto quant_param = input_tensor->quant_params();
